In [3]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>

In [4]:
############################################################
# pyupbit.get_current_price("KRW-BTC") : BIT 현재가
# --> price_KRW_BTC : BTC 현재가 
# upbit.get_balance(ticker="KRW-BTC") : BIT 잔고 조회 (My_Wallet_BIT)
# My_Wallet_BIT = upbit.get_balance(ticker="KRW-BTC") # <class 'float'>
# My_Wallet_Money = upbit.get_balance(ticker="KRW")
# My_Wallet_BITWON = float(price_KRW_BTC * My_Wallet_BIT))
############################################################
import os
import jwt
import uuid
import hashlib
import pyupbit
import datetime, time# for sleep
from urllib.parse import urlencode

import requests

access_key = os.environ['UPBIT_OPEN_API_ACCESS_KEY']
secret_key = os.environ['UPBIT_OPEN_API_SECRET_KEY']
server_url = os.environ['UPBIT_OPEN_API_SERVER_URL']
upbit = pyupbit.Upbit(access_key, secret_key)

payload = {
    'access_key': access_key,
    'nonce': str(uuid.uuid4()),
}

jwt_token = jwt.encode(payload, secret_key) #.decode('utf-8')
authorize_token = 'Bearer {}'.format(jwt_token)
headers = {"Authorization": authorize_token}

res = requests.get(server_url + "/v1/accounts", headers=headers)

# print(res.json())
# [{'currency': 'BTC', 'balance': '0.00026', 'locked': '0.0', 'avg_buy_price': '54041000', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}]

# print(pyupbit.get_ohlcv("KRW-BTC", interval="day", count=365))
Daily_Check = pyupbit.get_ohlcv("KRW-BTC", interval="day", count=365)
Daily_Check['diff1'] = round(((Daily_Check['high'] - Daily_Check['low']) / Daily_Check['low']) * 100,2)
Daily_Check['diff2'] = round(((Daily_Check['open'] - Daily_Check['close']) / Daily_Check['open']) * 100,2)
Daily_Check_Minus = Daily_Check[Daily_Check['diff2'] < 0]
print("# 마이너스만..")
print(Daily_Check_Minus['diff2'].max())
print(Daily_Check_Minus['diff2'].min())
print(Daily_Check_Minus['diff2'].mean() * 2)
Lower_Value_V = round(Daily_Check_Minus['diff2'].mean() * 2,1)

Upper_Value = float(1.5) # 수익라인, UPBit 거래수수료
# Lower_Value = float(-1.5)
Lower_Value = float(Lower_Value_V)

price_KRW_BTC = pyupbit.get_current_price("KRW-BTC") # BIT 현재가
Min_Call_Price = float(5000)
Min_Call_Volume = 5000 / price_KRW_BTC # 5000원을 현대 BIT가로 나눈값

def Call_Condition(File_Name) :

    payload = {
        'access_key': access_key,
        'nonce': str(uuid.uuid4()),
    }

    jwt_token = jwt.encode(payload, secret_key) #.decode('utf-8')
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.get(server_url + "/v1/accounts", headers=headers)

    # 자산 종류 수로 내가 가진 자산을 확인 2이면 BIT와 KRW, 1개면 KRW로 판단 (BIT없음)
    # ---> 1개일 경우 종류를 확인해봐야할 듯... (2021.03.30)
    if len(res.json()) == 2 :
        My_Wallet_KRW = res.json()[0] # {'error': {'message': '인증된 IP가 아닙니다.', 'name': 'no_authorization_i_p'}}
        My_Wallet_BIT = res.json()[1]

        print("# MY_Wallet_KRW : ")
        print(My_Wallet_KRW)
        print("# MY_Wallet_BIT : ")
        print(My_Wallet_BIT)
    elif len(res.json()) == 1 :
        My_Wallet_KRW = res.json()[0]

        print("# MY_Wallet_KRW : ")
        print(My_Wallet_KRW)
        print("# BIT가 없으므로 최초 매수")

        __Make_Put("BUY", 5000, 5000, OneDay_BUY_Count, File_Name) # 조건, 최소금액, 최소BIT, File_Name
    else :
        print("# res.json LENGTH is Wrong")


    ############################################################
    Upper_Value = float(1.5) # 수익라인, UPBit 거래수수료
    Lower_Value = float(-15) # 물타기라인, UPBit 거래수수료
    # Min_Call_Price = float(5000)
    # price_KRW_BTC = pyupbit.get_current_price("KRW-BTC") # BIT 현재가
    # Min_Call_Volume = 5000 / price_KRW_BTC
    print("# 최소매수가능 금액 : %f" % Min_Call_Price)
    print("# 최소매수가능 BIT : %f" % Min_Call_Volume)
    ############################################################

    ############################################################
    Call_Price_Possible = float(My_Wallet_KRW['balance']) / 10 # 보유 금액의 1/10
    # Call_Price_Possible = float(price_KRW_BTC * My_Wallet_BIT) / 10
    Call_Volume_Possible = upbit.get_balance(ticker="KRW-BTC") / 10 # 보유 BIT의 1/10
    print("# 1/10 Volume = %f" % Call_Volume_Possible)
    output.write("# 1/10 Volume = %f\n" % Call_Volume_Possible)
    print("# 1/10 Price = %f" % Call_Price_Possible)
    output.write("# 1/10 Price = %f\n" % Call_Price_Possible)
    print("")
    output.write("\n")
    ############################################################

    if Call_Price_Possible < Min_Call_Price : # 가진 자산의 1/10을 매도하거나 매수하지만 기본금액에 모자라면 기본금액으로 설정
        i = 1
        Temp_Possible = 0
        print("Min_Call_Price : %s" % Min_Call_Price)
        print("Min_Call_Price : %s" % type(Min_Call_Price)) # <class 'float'>
        print("Call_Price_Possbile : %s" % Call_Price_Possible)
        print("Call_Price_Possbile : %s" % type(Call_Price_Possible)) # <class 'float'>
        while Call_Price_Possible < Min_Call_Price : # 현재가 변동으로 10%를 추가로 잡아줌
            if Temp_Possible == 0 : 
                Temp_Possible = Call_Price_Possible
            Call_Price_Possible = Temp_Possible * i
            print("# %s번째 Price : %s" % (i, Call_Price_Possible))
            output.write("# %s번째 Price : %s\n" % (i, Call_Price_Possible))
            i += 1
            if i > 10 :
                Call_Price_Possible = Min_Call_Price

        print("# 완료 %s번째 Price : %s" % (i, Call_Price_Possible))
        output.write("# 완료 %s번째 Price : %s\n" % (i, Call_Price_Possible))
        print("")
        output.write("\n")

    # Temp_Volume = Call_Volume_Possible * price_KRW_BTC
    if Call_Volume_Possible < Min_Call_Volume : # 가진 자산의 1/10을 매도하거나 매수하지만 기본금액에 모자라면 기본금액으로 설정
        i = 1
        while (Call_Volume_Possible * price_KRW_BTC) < Min_Call_Price :
            print("# %s번째 Volume : %f" % (i, Call_Volume_Possible))
            output.write("# %s번째 Volume : %f\n" % (i, Call_Volume_Possible))
            Call_Volume_Possible = Call_Volume_Possible * i
            # if i < 0 :
            #     Call_Volume_Possible = price_KRW_BTC / Min_Call_Price
            #     break
            i += 1
        print("# 완료 %s번째 Volume : %f" % (i, Call_Volume_Possible))
        output.write("# 완료 %s번째 Volume : %f\n" % (i, Call_Volume_Possible))
        print("")
        output.write("\n")

    print("# 최종가격대")
    output.write("# 최종가격대\n")
    print("# Call_Price_Possible = %f" % Call_Price_Possible)
    output.write("# Call_Price_Possible = %f\n" % Call_Price_Possible)
    print("# Call_Volume_Possible = %f" % Call_Volume_Possible)
    output.write("# Call_Volume_Possible = %f\n" % Call_Volume_Possible)
    print("")
    output.write("\n")

    output.close()

    return Call_Price_Possible, Call_Volume_Possible

def __Make_OrderLists(SIDE) :

    if SIDE == 'SELL' : # bid : 매수, ask : 매도
        query = {
        'market': 'KRW-BTC',
        'state': 'done', # 주문 상태 , wait : 체결 대기 (default),  done : 전체 체결 완료,  cancel : 주문 취소
        }
    elif SIDE == 'BUY' : # bid : 매수, ask : 매도
        query = {
        'market': 'KRW-BTC',
        'state': 'cancel', # 주문 상태 , wait : 체결 대기 (default),  done : 전체 체결 완료,  cancel : 주문 취소
        }
    else : # bid : 매수, ask : 매도
        print("# 주문내역 조회 : 오류")
        # ---> 매수는 cancel 로 잡히고 매도는 done으로 잡힌다
        # ---> 매수 시 시장가로 매수하니 소수점이하..어쩌고 잔량이 어쩌고..
    # query_string = urlencode(query)
    query_string = urlencode(query).encode()

    # print("# Query : ") # {'error': {'message': 'Jwt의 query를 검증하는데 실패하였습니다.', 'name': 'invalid_query_payload'}}
    # print(query)

    m = hashlib.sha512()
    m.update(query_string)
    query_hash = m.hexdigest()

    payload = {
        'access_key': access_key,
        'nonce': str(uuid.uuid4()),
        'query_hash': query_hash,
        'query_hash_alg': 'SHA512',
    }

    jwt_token = jwt.encode(payload, secret_key)
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.get(server_url + "/v1/orders", params=query, headers=headers)
    print("# res.json()")
    print(len(res.json()))

    Now_DATE = datetime.datetime.today().strftime("%Y-%m-%d")
    for res_list in range(len(res.json())) :
    #     print(type(res.json()[res_list])) # <class 'dict'>
    #     print("res.json()[%s]" % res_list)
    #     # print(res.json()[res_list])
    #     print(res.json()[res_list]["uuid"])
    #     print(res.json()[res_list]["side"])
    #     print(res.json()[res_list]["ord_type"])
    #     print(res.json()[res_list]["state"])
        print(res.json()[res_list]["created_at"][:10]) # <class 'str'>
    #     Today_Trade = res.json()[res_list]["created_at"][:10]
        # print(Today_Trade)
        # if Today_Trade == Now_DATE[:10] :
        #     print("Today trade exist")
        # else :
        #     print("Today trade NONONON")

    Trade_List_uuid, Trade_List_side = [], []
    for res_dict in res.json() : # <class 'dict'>
        for keys, values in res_dict.items() :
            DATE_HIST, SIDE_HIST = 0, 0
            if keys == 'created_at' :
                # print(values[:10]) # <class 'str'> 2021-02-20T10:41:25+09:00
                # print(Now_DATE)
                if values[:10] == Now_DATE :
                    Trade_List_uuid.append(res_dict['uuid'])
                    Trade_List_side.append(res_dict['side'])
    
    print(Trade_List_uuid)
    print(Trade_List_side)

    Detail_Trade(Trade_List_uuid)

    if SIDE == 'SELL' : # bid : 매수, ask : 매도
        SIDE = 'ask'
        if SIDE in Trade_List_side :
            print("# 주문내역 조회 : 매도 있음")
            return SIDE
    elif SIDE == 'BUY' : # bid : 매수, ask : 매도
        SIDE = 'bid'
        if SIDE in Trade_List_side :
            print("# 주문내역 조회 : 매수 있음")
            return SIDE
    else : # bid : 매수, ask : 매도
        print("# 주문내역 조회 : 오류")
        return None

        # {'uuid': '418f0313-e001-4cc3-9dcc-f656dd24f3b2', uuid	주문의 고유 아이디	String
        #  'side': 'ask', side	주문 종류	String
        #  'ord_type': 'market', ord_type	주문 방식	String
        #  'price': None, price	주문 당시 화폐 가격	NumberString
        #  'state': 'done', state	주문 상태	String
        #  'market': 'KRW-BTC', market	마켓의 유일키	String
        #  'created_at': '2021-02-20T10:40:42+09:00', created_at	주문 생성 시간	DateString
        #  'volume': '0.000078', volume	사용자가 입력한 주문 양	NumberString
        #  'remaining_volume': '0.0', remaining_volume	체결 후 남은 주문 양	NumberString
        #  'reserved_fee': '0.0', reserved_fee	수수료로 예약된 비용	NumberString
        #  'remaining_fee': '0.0', remaining_fee	남은 수수료	NumberString
        #  'paid_fee': '2.509884', paid_fee	사용된 수수료	NumberString
        #  'locked': '0.0', locked	거래에 사용중인 비용	NumberString
        #  'executed_volume': '0.000078', executed_volume	체결된 양	NumberString
        #  'trades_count': 1} trade_count	해당 주문에 걸린 체결 수	Integer

def Detail_Trade(Trade_List_uuid) :

    print("# Detail_Trace Funtion START")

    for UUID in Trade_List_uuid :
        query = {
            'uuid': UUID,
        }
        query_string = urlencode(query).encode()

        m = hashlib.sha512()
        m.update(query_string)
        query_hash = m.hexdigest()

        payload = {
            'access_key': access_key,
            'nonce': str(uuid.uuid4()),
            'query_hash': query_hash,
            'query_hash_alg': 'SHA512',
        }

        jwt_token = jwt.encode(payload, secret_key)
        authorize_token = 'Bearer {}'.format(jwt_token)
        headers = {"Authorization": authorize_token}

        res = requests.get(server_url + "/v1/order", params=query, headers=headers)

        # print(res.json())
        Detail_Trade_Value = res.json()
        print(Detail_Trade_Value['side'])

def __Make_Put(CALL, Call_Possible, Min_Call_Price, OneDay_BUY_Count, File_Name) :

    # Call_Possible 값은 <--- My_Wallet_BIT = upbit.get_balance(ticker="KRW-BTC") # 자산 내 BIT
    # Min_Call_Price = float(5000)

    print("# __Make_Put Function Start")
    output = open(File_Name, "a")

    print("# %s : CALL Volume/Price : %f개 or 원" % (datetime.datetime.today().strftime("%H:%M:%S"), Call_Possible))
    output.write("# %s : CALL Volume/Price : %f개 or 원\n" % (datetime.datetime.today().strftime("%H:%M:%S"), Call_Possible))
    # 시장가 주문은 ord_type 필드를 price or market 으로 설정해야됩니다.
    if CALL == "SELL" :
        # 매도 주문의 경우 ord_type을 market로 설정하고 price을 null 혹은 제외해야됩니다.
        # Call_Possible = upbit.get_balance(ticker="KRW-BTC") # 매도할 때는 전체를 매도하니간 굳이 계산없이 전체
        print("# %s : BIT 잔고 조회 : %f" % (datetime.datetime.today().strftime("%H:%M:%S"), upbit.get_balance(ticker="KRW-BTC"))) # 특정 코인이나 원화의 잔고만 조회
        output.write("# %s : BIT 잔고 조회 : %f\n" % (datetime.datetime.today().strftime("%H:%M:%S"), upbit.get_balance(ticker="KRW-BTC"))) # 특정 코인이나 원화의 잔고만 조회
        price_KRW_BTC = pyupbit.get_current_price("KRW-BTC") # <class 'float'> ---> BIT가격은 계속 바뀔테니 다시 조회
        TEMP_Call_Possible = Call_Possible * price_KRW_BTC # 현재 보유 BIT개수 * 현재 BIT값
        if TEMP_Call_Possible < Min_Call_Price : # 최소금액에 못미치면 그냥 returne
            print("# %s :---> 매도 Bit가 최소금액이 안됨" % (datetime.datetime.today().strftime("%H:%M:%S")))
            output.write("# %s :---> 매도 Bit가 최소금액이 안됨\n" % (datetime.datetime.today().strftime("%H:%M:%S")))
            return None
        else :
            print("# %s : Make call SELL" % datetime.datetime.today().strftime("%H:%M:%S"))
            output.write("# %s : Make call SELL\n" % datetime.datetime.today().strftime("%H:%M:%S"))
            CALL = "ask" # 매도
            ORD_TYPE = "market"
            PRICE = ''
            VOLUME = Call_Possible
    elif CALL == "BUY" :
        OneDay_BUY_Count += 1
        if OneDay_BUY_Count > 2 :
            return None
        # 매수 주문의 경우 ord_type을 price로 설정하고 volume을 null 혹은 제외해야됩니다.
        print("# %s : Make call BUY" % datetime.datetime.today().strftime("%H:%M:%S"))
        output.write("# %s : Make call BUY\n" % datetime.datetime.today().strftime("%H:%M:%S"))
        CALL = "bid" # 매수
        ORD_TYPE = "price"
        VOLUME = ''
        PRICE = Call_Possible
    else :
        print("# %s : Make Call ERROR~!" % datetime.datetime.today().strftime("%H:%M:%S"))
        output.write("# %s : Make Call ERROR~!\n" % datetime.datetime.today().strftime("%H:%M:%S"))

    query = {
        'market': 'KRW-BTC',
        'side': CALL, # 주문 종류 (필수) - bid : 매수, ask : 매도
        'volume': VOLUME, # 주문 수량, 주문량 (지정가, 시장가 매도 시 필수) NumberString
        'price': PRICE, # 유닛당 주문 가격
        'ord_type': ORD_TYPE,
        # 주문 타입 (필수) - limit : 지정가 주문, price : 시장가 주문(매수), market : 시장가 주문(매도)
        # identifier : 조회용 사용자 지정값 (선택) String (Uniq 값 사용)
    }
    query_string = urlencode(query).encode()

    print("# Query : ") # {'market': 'KRW-BTC', 'side': 'bid', 'volume': '', 'price': 6689.808521999999, 'ord_type': 'price'}
    print(query)

    m = hashlib.sha512()
    m.update(query_string)
    query_hash = m.hexdigest()

    payload = {
        'access_key': access_key,
        'nonce': str(uuid.uuid4()),
        'query_hash': query_hash,
        'query_hash_alg': 'SHA512',
    }

    jwt_token = jwt.encode(payload, secret_key) # .decode('utf-8')
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    # print(jwt_token)
    # print(authorize_token)
    # print(headers)
    res = requests.post(server_url + "/v1/orders", params=query, headers=headers)

    print("# __Make_Put : ")
    output.write("# __Make_Put : \n")
    print(type(res.text)) # <class 'str'>
    print(res.text)
     # 매수 : {"error":{"message":"주문가능한 금액(BTC)이 부족합니다.","name":"insufficient_funds_ask"}}
     # 매수 : {"uuid":"9ddd3561-1188-4e5a-83fb-5810bb7f0c94","side":"bid","ord_type":"price","price":"6689.8","state":"wait","market":"KRW-BTC","created_at":"2021-02-27T09:00:29+09:00","volume":null,"remaining_volume":null,"reserved_fee":"3.3449","remaining_fee":"3.3449","paid_fee":"0.0","locked":"6693.1449","executed_volume":"0.0","trades_count":0}
    output_res_text = res.text
    output.write(output_res_text)

    output.close()

def Current_Query(OneDay_BUY_Count, File_Name) :

    print("# Current_Query Function Start")

    payload = {
        'access_key': access_key,
        'nonce': str(uuid.uuid4()),
    }

    jwt_token = jwt.encode(payload, secret_key) #.decode('utf-8')
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.get(server_url + "/v1/accounts", headers=headers)

    # print(res.json())
    if len(res.json()) < 2 :
        print("# 보유자산 1개 뿐임 (BIT or WON)")
        Temp_Check = res.json()[0]
        if Temp_Check['currency'] == 'KRW' :
            print("# 보유자산 1개 = WON만 ---> 바로 주문)")
            __Make_Put("BUY", 5000, 5000, OneDay_BUY_Count, File_Name) # 조건, 최소금액, 최소BIT, File_Name
            return None
        elif Temp_Check['currency'] == 'BTC' :
            print("# 보유자산 1개 = BIT만)")
            return None
        else :
            print("# 보유자산 Error")
            return None

    My_Wallet_KRW = res.json()[0]
    My_Wallet_BIT = res.json()[1]
    # {'currency': 'KRW', 'balance': '1718.88109746', 'locked': '0.0', 'avg_buy_price': '0', 'avg_buy_price_modified': True, 'unit_currency': 'KRW'}
    # {'currency': 'BTC', 'balance': '0.00033108', 'locked': '0.0', 'avg_buy_price': '60728083.3', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}

    print(My_Wallet_KRW)
    # {'currency': 'KRW', 'balance': '1718.88109746', 'locked': '0.0', 'avg_buy_price': '0', 'avg_buy_price_modified': True, 'unit_currency': 'KRW'}
    print(My_Wallet_KRW["avg_buy_price"])
    print(My_Wallet_BIT)
    # {'currency': 'BTC', 'balance': '0.00033108', 'locked': '0.0', 'avg_buy_price': '60728083.3', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
    print(My_Wallet_BIT["avg_buy_price"])

    price_KRW_BTC = pyupbit.get_current_price("KRW-BTC") # <class 'float'>
    print("############################################################")
    output.write("############################################################\n")
    print("# BTC 현재가 : %s 원" % price_KRW_BTC)
    output.write("# KRW-BTC 현재가 : %s 원\n" % price_KRW_BTC)
    print("# BIT 잔고 조회 : %s" % My_Wallet_BIT["balance"]) # 특정 코인이나 원화의 잔고만 조회
    output.write("# KRW-BTC 현재가 : %s 원\n" % price_KRW_BTC)
    # My_Wallet_BIT = upbit.get_balance(ticker="KRW-BTC") # <class 'float'>
    BIT_WON = float(My_Wallet_BIT["balance"]) * price_KRW_BTC
    print("# 현재 BIT원화 잔고 : %f 원" % BIT_WON)
    output.write("# 현재 BIT원화 잔고 : %f 원" % BIT_WON)
    My_Wallet_KRW = My_Wallet_KRW["balance"]
    print("# 현재 원화 잔고 조회 : %s" % My_Wallet_KRW) # 특정 코인이나 원화의 잔고만 조회
    output.write("# 현재 원화 잔고 조회 : %s\n" % My_Wallet_KRW) # 특정 코인이나 원화의 잔고만 조회
    print("############################################################")
    output.write("############################################################\n")

    # My_Wallet_BIT = My_Wallet_BIT
    # print("My_Wallet : %s" % My_Wallet)
    # My_Wallet : {'currency': 'BTC', 'balance': '0.00033108', 'locked': '0.0', 'avg_buy_price': '60728083.3', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}

    ############################################################
    # print(type(My_Wallet_BIT))
    # print(My_Wallet_BIT)
    print("# 평단가 : %s" % My_Wallet_BIT['avg_buy_price']) # <class 'str'>
    output.write("# 평단가 : %s\n" % My_Wallet_BIT['avg_buy_price']) # <class 'str'>
    print("# 현재가 : %f" % pyupbit.get_current_price("KRW-BTC")) # <class 'float'>
    output.write("# 현재가 : %f\n" % pyupbit.get_current_price("KRW-BTC")) # <class 'float'>
    ############################################################

    ############################################################
    TEMP_My_Wallet = float(My_Wallet_BIT['avg_buy_price'])
    My_Wallet_BIT = upbit.get_balance(ticker="KRW-BTC") # <class 'float'>
    TEMP_Current = pyupbit.get_current_price("KRW-BTC")
    if int(TEMP_My_Wallet) <= 0 :
        print("# TEMP My Wallet : %f ---> 보유 BIT가 없음" % TEMP_My_Wallet)
        output.write("# TEMP My Wallet : %f ---> 보유 BIT가 없음\n" % TEMP_My_Wallet)
        return TEMP_My_Wallet
    TEMP_Percent = (TEMP_Current - TEMP_My_Wallet) / TEMP_My_Wallet * 100
    TEMP_Percent = round(TEMP_Percent, 2)
    print("# 현재 수익률 : %f " % TEMP_Percent) # <class 'float'> 
    # time.sleep(60)
    ############################################################

    return TEMP_My_Wallet

def Start_BIT_Trade(Call_Price_Possible, Call_Volume_Possible, File_Name) :

    OneDay_BUY_Count = 0 # 하루에 매수하는 회수를 제어하기 위한 Count, PUT(BUY) 호출할 때마다 1 Count (Start_BIT_Trade에서 최초 0부터 해서 시작)
    infinite_while = 1

    while infinite_while == 1 :
        output = open(File_Name, "a")

        # print("\n# Time Check : %s" % (datetime.datetime.today().strftime("%H:%M:%S")))
        # output.write("\n# Time Check : %s\n" % (datetime.datetime.today().strftime("%H:%M:%S")))
        now_time = datetime.datetime.today().strftime("%H:%M:%S")

        ############################################################
        # Loop 안에서 새롭게 조회를 해야 한다
        payload = {
            'access_key': access_key,
            'nonce': str(uuid.uuid4()),
        }

        jwt_token = jwt.encode(payload, secret_key) #.decode('utf-8')
        authorize_token = 'Bearer {}'.format(jwt_token)
        headers = {"Authorization": authorize_token}

        res = requests.get(server_url + "/v1/accounts", headers=headers)

        if len(res.json()) < 2 :
            print("# 보유자산 1개 뿐임 (BIT or WON)")
            Temp_Check = res.json()[0]
            if Temp_Check['currency'] == 'KRW' :
                print("# 보유자산 1개 = WON만 ---> 바로 주문)")
                __Make_Put("BUY", 5000, 5000, OneDay_BUY_Count, File_Name) # 조건, 최소금액, 최소BIT, File_Name
                continue
            elif Temp_Check['currency'] == 'BTC' :
                print("# 보유자산 1개 = BIT만)")
                continue
            else :
                print("# 보유자산 Error")
                continue

        My_Wallet_KRW = res.json()[0]
        My_Wallet_BIT = res.json()[1]
        # print(My_Wallet_KRW)
        # print(My_Wallet_BIT)
        price_KRW_BTC = pyupbit.get_current_price("KRW-BTC") # <class 'float'>
        ############################################################

        # My_Wallet = upbit.get_balances(0)[0]
        TEMP_My_Wallet = float(My_Wallet_BIT['balance']) # 내 보유 BIT
        TEMP_Current = pyupbit.get_current_price("KRW-BTC") # 현재 BIT가격
        # print("Temp_My_Wallet : %f" % TEMP_My_Wallet)
        # print("Temp_Current : %f" % TEMP_Current)

        # TEMP_Wallet_BIT = float(My_Wallet_BIT['balance']) # 내 보유 BIT
        My_Wallet_Money = float(My_Wallet_KRW['balance']) # 내 보유 WON

        TEMP_Percent = ((TEMP_Current - float(My_Wallet_BIT['avg_buy_price'])) / float(My_Wallet_BIT['avg_buy_price'])) * 100

        # TEMP_Percent = (TEMP_Current - TEMP_My_Wallet) / TEMP_My_Wallet * 100
        # TEMP_Percent = (TEMP_Current - TEMP_My_Wallet) / TEMP_My_Wallet * 100
        TEMP_Percent = round(TEMP_Percent, 2)
        # print("TEMP_Percent : %s" % TEMP_Percent)

        # TEMP_BITWON = float(TEMP_Current * TEMP_My_Wallet)
        TEMP_BITWON = upbit.get_balance(ticker="KRW-BTC") # <class 'float'>

        # if TEMP_BITWON < Min_Call_Price : # 현재 보유BIT가 5000원을 넘지 않을 경우 매수, Min_Call_Price = float(5000)
        if TEMP_BITWON < 0 : # 0보다 작다면 없는 것으로 판단하고, 소수점이라도 있으면 있다고 판단하여 없을 경우에 매수 호출
            My_Wallet_BIT = upbit.get_balance(ticker="KRW-BTC") # 자산 내 BIT ---> 자산 내 BIT수가 바뀌진 않지...
            __Make_Put("BUY", 5000, 5000, OneDay_BUY_Count, File_Name)

        if TEMP_Percent > Upper_Value :
            print("# %s : 조건 OK ! ---> 수익 조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 BIT는 %s, KRW-BIT가격: %s" % \
            (datetime.datetime.today().strftime("%H:%M:%S"), Upper_Value, Lower_Value, TEMP_Percent, TEMP_BITWON, price_KRW_BTC))
            output.write("# %s : 조건 OK ! ---> 수익 조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 BIT는 %s, KRW-BIT가격: %s\n" % \
            (datetime.datetime.today().strftime("%H:%M:%S"), Upper_Value, Lower_Value, TEMP_Percent, TEMP_BITWON, price_KRW_BTC))
            print("# %s : %s가 %s 보다 크다 ---> SELL!" % (datetime.datetime.today().strftime("%H:%M:%S"), TEMP_Percent, Upper_Value))
            output.write("# %s : %s가 %s 보다 크다 ---> SELL!\n" % (datetime.datetime.today().strftime("%H:%M:%S"), TEMP_Percent, Upper_Value))
            # __Check_Call()
            # print("# Make Put SELL : %s개 매도" % Call_Volume_Possible)
            # output.write("# Make Put SELL : %s개 매도\n" % Call_Volume_Possible)
            # __Make_Put("SELL", Call_Volume_Possible, Min_Call_Price, OneDay_BUY_Count, File_Name)
            print("# %s : Make Put SELL : %s개 매도" % (datetime.datetime.today().strftime("%H:%M:%S"), My_Wallet_BIT)) # ---> 매도 때는 전체 bit 매도
            output.write("# %s : Make Put SELL : %s개 매도\n" % (datetime.datetime.today().strftime("%H:%M:%S"), My_Wallet_BIT))
            print("# testtest : %s" % My_Wallet_BIT)

            # 22:34:48 : Make Put SELL : {'currency': 'BTC', 'balance': '0.00007273', 'locked': '0.0', 'avg_buy_price': '68745000', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}개 매도
            # testtest : {'currency': 'BTC', 'balance': '0.00007273', 'locked': '0.0', 'avg_buy_price': '68745000', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}

            # Trade_Count = __Make_OrderLists("SELL") # ---> Return 값 : bid : 매수, ask : 매도
            My_Wallet_BIT = upbit.get_balance(ticker="KRW-BTC") # 자산 내 BIT ---> 자산 내 BIT수가 바뀌진 않지...
            __Make_Put("SELL", My_Wallet_BIT, Min_Call_Price, OneDay_BUY_Count, File_Name)
            # ---> Call_Price_Possible = float(My_Wallet_KRW['balance']) / 10 # 보유 금액의 1/10 계산값
            # ---> Call_Possible 값은 <--- My_Wallet_BIT = upbit.get_balance(ticker="KRW-BTC") # 자산 내 BIT
            # ---> Min_Call_Price = float(5000)
            
        elif TEMP_Percent < Lower_Value :
            print("# %s : 조건 OK ! ---> 수익 조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 BIT는 %s, KRW-BIT가격: %s" % \
            (datetime.datetime.today().strftime("%H:%M:%S"), Upper_Value, Lower_Value, TEMP_Percent, TEMP_BITWON, price_KRW_BTC))
            output.write("# %s : 조건 OK ! ---> 수익 조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 BIT는 %s, KRW-BIT가격: %s\n" % \
            (datetime.datetime.today().strftime("%H:%M:%S"), Upper_Value, Lower_Value, TEMP_Percent, TEMP_BITWON, price_KRW_BTC))
            if My_Wallet_Money < Min_Call_Price : # My_Wallet_KRW = res.json()[0] ---> 내 보유 WON, Min_Call_Price = float(5000)
                print("# %s : 현재 잔액이 5,000원보다 적다" % datetime.datetime.today().strftime("%H:%M:%S"))
                output.write("# %s : 현재 잔액이 5,000원보다 적다\n" % datetime.datetime.today().strftime("%H:%M:%S"))
            else :
                print("# %s : BUY~!" % datetime.datetime.today().strftime("%H:%M:%S"))
                output.write("# %s : BUY~!\n" % datetime.datetime.today().strftime("%H:%M:%S"))
                print("# %s : Make Put SELL : %s원 매수" % (datetime.datetime.today().strftime("%H:%M:%S"), Call_Price_Possible))
                output.write("# %s : Make Put SELL : %s원 매수\n" % (datetime.datetime.today().strftime("%H:%M:%S"), Call_Price_Possible))

                Trade_Count = __Make_OrderLists("BUY") # ---> Return 값 : bid : 매수, ask : 매도
                if "bid" in Trade_Count :
                    print("# %s : 매수 건 존재" % datetime.datetime.today().strftime("%H:%M:%S"))
                    output.write("# %s : 매수 건 존재\n" % datetime.datetime.today().strftime("%H:%M:%S"))
                else :
                    print("# %s : 매수 건 없음" % datetime.datetime.today().strftime("%H:%M:%S"))
                    output.write("# %s : 매수 건 없음" % datetime.datetime.today().strftime("%H:%M:%S"))
                    __Make_Put("BUY", Call_Price_Possible, Min_Call_Price, OneDay_BUY_Count, File_Name)
                    # ---> Call_Price_Possible = float(My_Wallet_KRW['balance']) / 10 # 보유 금액의 1/10 계산값
                    # ---> Call_Possible 값은 <--- My_Wallet_BIT = upbit.get_balance(ticker="KRW-BTC") # 자산 내 BIT
                    # ---> Min_Call_Price = float(5000)
        else :
            price_KRW_BTC = pyupbit.get_current_price("KRW-BTC")
            print("# %s : 조건에 안들어온다 WAIT! ---> 수익 조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 BIT는 %f, KRW-BIT가격: %s" % \
            (datetime.datetime.today().strftime("%H:%M:%S"), Upper_Value, Lower_Value, TEMP_Percent, TEMP_BITWON, price_KRW_BTC))
            output.write("# %s : 조건에 안들어온다 WAIT! ---> 수익 조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 BIT는 %f, KRW-BIT가격: %s\n" % \
            (datetime.datetime.today().strftime("%H:%M:%S"), Upper_Value, Lower_Value, TEMP_Percent, TEMP_BITWON, price_KRW_BTC))

        # ret = upbit.get_orders('KRW-XRP', 'wait', 5, 'asc')
        # ret = upbit.get_order('KRW-BTC', 'done')
        output.close()
        time.sleep(60)

    # print("# Manual Call Start")
    # __Make_Put("BUY")
    # print("# Manual Call End")

if __name__ == '__main__' :

    # start_time = "09:00"
    # stop_time = "15:30"

    OneDay_BUY_Count = 0 # 하루에 매수하는 회수를 제어하기 위한 Count, PUT(BUY) 호출할 때마다 1 Count (Start_BIT_Trade에서 최초 0부터 해서 시작)

    DateTime = datetime.datetime.today().strftime("%Y%m%d")
    File_Name = "D:\\Python\\Log\\UPBit_History"+"_"+DateTime+".txt"
    output = open(File_Name, "a")
    output.write("# -*- coding: utf-8 -*-\n")

    # Current_Query(File_Name)
    Temp_My_Wallet = Current_Query(OneDay_BUY_Count, File_Name)
    # print(Temp_My_Wallet)
    # print(type(Temp_My_Wallet)) # <class 'float'>
    
    if int(Temp_My_Wallet) <= 0 :
        print("# 보유 자산 개수가 작다 or BIT가가 0보다 작다")
        print(Temp_My_Wallet)
        __Make_Put("BUY", 5000, 5000, OneDay_BUY_Count, File_Name) # 조건, 최소금액, 최소BIT, File_Name
    else :
        print("# 보유 자산이 0보다 크다")

    Call_Price_Possible, Call_Volume_Possible = Call_Condition(File_Name)
    # print("TEST1")
    # __Make_OrderLists("SELL") # ---> 매도만 나온다..(2021.03.28)
    # print("TEST2")
    # __Make_OrderLists("BUY") # ---> 매도만 나온다..(2021.03.28)
    Start_BIT_Trade(Call_Price_Possible, Call_Volume_Possible, File_Name)
    # print(pyupbit.get_current_price("KRW-BTC")) # <class 'float'> ---> BIT가격은 계속 바뀔테니 다시 조회
    # My_Wallet_Money = float(My_Wallet_KRW['balance']) # 내 보유 WON

#------> BIT 개수가 바뀌어?????????
# 21:55:27 : 조건에 안들어온다 WAIT! ---> 수익 조건 : 1.5, 물타기 조건 : -5.7, 현재는 0.04, 내 BIT는 5001.63702, KRW-BIT가격: 72499000.0
# 21:56:27 : 조건에 안들어온다 WAIT! ---> 수익 조건 : 1.5, 물타기 조건 : -5.7, 현재는 0.04, 내 BIT는 5001.63702, KRW-BIT가격: 72496000.0
# 21:57:28 : 조건에 안들어온다 WAIT! ---> 수익 조건 : 1.5, 물타기 조건 : -5.7, 현재는 0.04, 내 BIT는 5001.775000000001, KRW-BIT가격: 72501000.0
# 21:58:28 : 조건에 안들어온다 WAIT! ---> 수익 조건 : 1.5, 물타기 조건 : -5.7, 현재는 0.11, 내 BIT는 5005.01753, KRW-BIT가격: 72533000.0
# 21:59:28 : 조건에 안들어온다 WAIT! ---> 수익 조건 : 1.5, 물타기 조건 : -5.7, 현재는 0.16, 내 BIT는 5007.84612, KRW-BIT가격: 72588000.0
# 22:00:29 : 조건에 안들어온다 WAIT! ---> 수익 조건 : 1.5, 물타기 조건 : -5.7, 현재는 0.25, 내 BIT는 5012.3994600000005, KRW-BIT가격: 72654000.0
# 22:01:29 : 조건에 안들어온다 WAIT! ---> 수익 조건 : 1.5, 물타기 조건 : -5.7, 현재는 0.22, 내 BIT는 5010.53673, KRW-BIT가격: 72654000.0
# 22:02:29 : 조건에 안들어온다 WAIT! ---> 수익 조건 : 1.5, 물타기 조건 : -5.7, 현재는 0.25, 내 BIT는 5012.3994600000005, KRW-BIT가격: 72654000.0

# 마이너스만..
-0.07
-17.05
-5.789430894308943
# Current_Query Function Start
{'currency': 'KRW', 'balance': '17274.74669382', 'locked': '0.0', 'avg_buy_price': '0', 'avg_buy_price_modified': True, 'unit_currency': 'KRW'}
0
{'currency': 'BTC', 'balance': '0.00006309', 'locked': '0.0', 'avg_buy_price': '79250000', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
79250000
############################################################
# BTC 현재가 : 78653000.0 원
# BIT 잔고 조회 : 0.00006309
# 현재 BIT원화 잔고 : 4962.217770 원
# 현재 원화 잔고 조회 : 17274.74669382
############################################################
# 평단가 : 79250000
# 현재가 : 78653000.000000


KeyboardInterrupt: 